# 추천 시스템 공부 - 협업필터링
- '내용 기반 필터링'은 자기가 좋아하는 것과 유사한 속성을 가진 아이템을 추천해주는 방식
- reference : https://dacon.io/codeshare/6140?dtype=recent
- 협업필터링 : 사용자가 아이템에게 매긴 평점이나 정보를 바탕으로 비슷한 취향의 고객 정보를 이용하여 아직 이용하지 않은 아이템을 추천하는 방식

## 1. 최근접 이웃 협업 필터링
다시 두가지로 분류됨
- 사용자 기반 : 취향이 비슷한 TOP-N의 사용자들이 좋아하는 아이템을 추천하는 방식
- 아이템 기반 : 아이템을 좋아하는지/싫어하는지 평가 기준이 유사한 아이템을 추천하는 방식
=> 일반적으로 아이템 기반이 더 정화곧가 좋다고 함. 유사한 상품 구매하낟고 해서 취향 비슷하다고 판단하기 어려우므로.

In [3]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

In [4]:
# 이미지 넣기위해
from IPython.display import Image

In [5]:
data_path = r'C:\Users\SeoJeongBin\Downloads\Movielens_small\ml-latest-small'
movies = pd.read_csv(data_path + '/movies.csv')
ratings = pd.read_csv(data_path + '/ratings.csv')
print(movies.shape)
print(ratings.shape)

(9742, 3)
(100836, 4)


In [6]:
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [7]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


### 1. pivot 데이터프레임 생성
- 각 유저가 Item에 대해서 몇 점을 부여했는지 확인하기 위해 기존 df에서 피봇처리를 해야함
![pivot](./pivot.png)

In [8]:
ratings = ratings.iloc[:,[0,1,2]]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')
ratings_matrix.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- ratings와 movies 데이터프레임을 movieId 기준으로 join (공통칼럼 : movieId)
- 생성된 title 열 기준으로 pivot 수행하면 다음과 같은 결과를 얻음 

In [9]:
# title 칼럼을 얻기위해 movies와 join 하여 df 생성
rating_movies = pd.merge(ratings, movies, on='movieId')
rating_movies.head(3)

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [10]:
# columns='title' 로 title 컬럼으로 pivot 수행. 
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

# NaN 값을 모두 0 으로 변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 그냥 cosine 유사도 하면 이건 user 사이의 유사도가 측정되게 됨
- 따라서 transpose()를 통해 행,열을 바꿔준다 

In [11]:
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head(3) 

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. 영화 간 유사도 산출

In [12]:
from sklearn.metrics.pairwise import cosine_similarity 

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)
print(item_sim.shape)
print(type(item_sim))

(9719, 9719)
<class 'numpy.ndarray'>


In [13]:
# ndarray 형태를 보기좋게 df 형태로 바꾸고자 함
# cosine_similarity() 로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)
print(item_sim_df.shape)
item_sim_df.head(3)

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


##### 특정 영화와 유사한 영화 뽑아보기

In [14]:
movie_name = "Godfather, The (1972)"
item_sim_df[movie_name].sort_values(ascending=False)[:6] # 유사도 상위 영화 6개뽑음

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.821773
Goodfellas (1990)                            0.664841
One Flew Over the Cuckoo's Nest (1975)       0.620536
Star Wars: Episode IV - A New Hope (1977)    0.595317
Fargo (1996)                                 0.588614
Name: Godfather, The (1972), dtype: float64

### 3. 영화 추천 하기 
- 개인화된 영화 추천의 가장 큰 특징은 **아직 관람하지 않은 영화**를 추천한다는 것
- 따라서, 아이템 유사도 & 관람한 영화의 평점 데이터를 기반으로, 새롭게 모든 영화의 예측 평점을 계산한 후 => 높은 예측 평점을 가진 영화를 추천해야 함

#### 추천 수식
![recommend_function](./recommend_function.png)

In [16]:
def predict_rating(ratings_arr, item_sim_arr) :
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)]) 
    return ratings_pred
# ndarray.dot : 내적의미 (같은 인덱스 값끼리 곱하여 합산)
# .sum(axis=1) : 가로방향으로 계산

In [17]:
ratings_arr =ratings_matrix.values
item_sim_arr = item_sim_df.values
ratings_pred = predict_rating(ratings_arr, item_sim_arr)

ratings_pred

array([[0.07034471, 0.5778545 , 0.32169559, ..., 0.13602448, 0.29295452,
        0.72034722],
       [0.01826008, 0.04274424, 0.01886104, ..., 0.02452792, 0.01756305,
        0.        ],
       [0.01188449, 0.03027871, 0.06443729, ..., 0.00922874, 0.01041982,
        0.08450144],
       ...,
       [0.32443466, 1.02254119, 0.5984666 , ..., 0.53858621, 0.52763888,
        0.69887065],
       [0.00483488, 0.05359271, 0.02625119, ..., 0.0131077 , 0.01832826,
        0.03337679],
       [3.62830323, 1.51791811, 0.83366781, ..., 1.81609065, 0.56507537,
        0.57465402]])

In [19]:
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns = ratings_matrix.columns)
print(ratings_pred_matrix.shape)
ratings_pred_matrix.head(3)

(610, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501


#### 예측 평가 지표 MSE

In [20]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))

아이템 기반 모든 인접 이웃 MSE:  9.895354759094706


#### 모든 인접 이웃 기준 MSE가 아닌 TOP-N 유사도를 가진 데이터들에 대해서만 예측 평점 계산해보기

In [21]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행. 
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) 
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))        
    return pred

In [22]:
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))


# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

아이템 기반 인접 TOP-20 이웃 MSE:  3.694957479362603


TOP-N의 이웃들 기준으로는 훨씬 나은 수준을 보여줌 

### 실제 특정 사용자에게 영화 추천하기 

In [27]:
# 9번 사용자가 어떤 영화 좋아하는지 높은 평점 순으로 나열해보기
user_id_num = 9 
user_rating_id = ratings_matrix.loc[user_id_num, :]
user_rating_id.head(3)

title
'71 (2014)                                 0.0
'Hellboy': The Seeds of Creation (2004)    0.0
'Round Midnight (1986)                     0.0
Name: 9, dtype: float64

In [28]:
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10] # 내림차순 10개

title
Adaptation (2002)                                                                 5.0
Citizen Kane (1941)                                                               5.0
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    5.0
Producers, The (1968)                                                             5.0
Lord of the Rings: The Two Towers, The (2002)                                     5.0
Lord of the Rings: The Fellowship of the Ring, The (2001)                         5.0
Back to the Future (1985)                                                         5.0
Austin Powers in Goldmember (2002)                                                5.0
Minority Report (2002)                                                            4.0
Witness (1985)                                                                    4.0
Name: 9, dtype: float64

영화 추천을 위해, 이미 평점 매긴거말고 새로운 영화 중에서 추천할 수 있도록 get_unseen_movies() 함수 생성

In [31]:
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임! 대상 index를 추출하여 list 객체로 만듦
    already_seen = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듦. 
    movies_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함. 
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

아이템 기반 유사도로 평점이 부여된 데이터셋에서 해단 사용자가 관람하지 않은 영화들 중 예측 평점이 높은 순으로 추천해 보자 

In [29]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [32]:
# 사용자가 관람하지 않는 영화명 추출   
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천 
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies

,pred_score
title,
Shrek (2001),0.866202
Spider-Man (2002),0.857854
"Last Samurai, The (2003)",0.817473
Indiana Jones and the Temple of Doom (1984),0.816626
"Matrix Reloaded, The (2003)",0.800990
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.765159
Gladiator (2000),0.740956
"Matrix, The (1999)",0.732693
Pirates of the Caribbean: The Curse of the Black Pearl (2003),0.689591


## 2. 잠재요인 협업 필터링
- 기존 사용자-아이템 평점 매트릭스 (ratings_matrix) 속에 숨어 있는 잠재 요인을 추출
- 행렬 분해 : 대규모 다차원 행렬을 차원 감소 기법으로 분해하여 잠재 요인 추출
- 기존 평점 매트릭스는 희소 행렬 형태임 (0이 대부분인) => 저차원 밀집 행렬의 user-factor(잠재요인), item-factor로 분해하여 아직 평점 부여하지 않은 데이터에 대한 예측 평점을 생성함
- 예측 평점은 분해된 두개의 내적 곱으로 산출됨
- 행렬 분해에 대한 기법으로 SGD(확률적 경사하강법)를 쓸 수 있음 : 내적 곱을 통해 예측 행렬을 계산하고 오류값을 업데이트하면 근사화 시키는 것임.
![matrix](./matrix.png)